[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinichiromizuno/QueryMultiTopic/blob/master/Lead.ipynb)

In [ ]:
# 作業ディレクトリの指定
work_dir =  '/content/drive/MyDrive/work_lead'
source_dir = '/content/drive/MyDrive/work_proposed'

In [ ]:
# Step1~4はProposedと同じ

In [ ]:
import json
import os
import random
from glob import glob

tgt_list_valid = []
tgt_list_test = []
for data in sorted(glob(source_dir + '/json_data_labeled/*')):
  filetype = data.split('/')[-1].split('.')[1]
  json_open = open(data)
  json_load = json.load(json_open)
  if filetype == 'valid':
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_valid.append(tgt_list)
  elif filetype == 'test':
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_test.append(tgt_list)

In [ ]:
def lead_list_creation_valid(lead_length_range, tgt_list_valid):
  lead_list_valid = []
  for tgt_list in tgt_list_valid:
    lead_list = []
    for lead_length in lead_length_range:
      lead_list_by_length = []
      for tgt_list_i in tgt_list:
        if lead_length <= len(tgt_list_i):
          one_list = [1] * lead_length
          zero_list = [0] * (len(tgt_list_i) - lead_length)
        else:
          one_list = [1] * len(tgt_list_i)
          zero_list = []
        lead_list_by_length.append(one_list + zero_list)
      lead_list.append(lead_list_by_length)
    lead_list_valid.append(lead_list)
  return lead_list_valid

In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid(all_predited_list, all_tgt_list, thres_list):
  best_all_predited_list = []
  for i in range(17):
    by_thres_predicted = []
    f1_list, acc_list = [], []
    for j, thres in enumerate(thres_list):
      by_qry_predicted = []
      by_qry_tgt = []
      for doc_pred, doc_tgt in zip(all_predited_list, all_tgt_list):
        by_qry_predicted += doc_pred[j][i]
        by_qry_tgt += doc_tgt[i]
      f1 = f1_score(by_qry_predicted, by_qry_tgt)
      acc = accuracy_score(by_qry_predicted, by_qry_tgt)
      f1_list.append(f1)
      acc_list.append(acc)
    bs = np.argmax(f1_list)
    print(f'Goal{i+1}, threshold: {thres_list[bs]}, f1_score: {round(f1_list[bs], 3)}, accuracy: {round(acc_list[bs], 3)}')
    best_all_predited_list.append([doc_pred[bs][i] for doc_pred in all_predited_list])
  
  best_all_predited_list_t = np.array(best_all_predited_list).T.tolist()
  single_predicted = []
  single_tgt = []
  for doc_pred, doc_tgt in zip(best_all_predited_list_t, all_tgt_list):
    for qry_pred, qry_tgt in zip(doc_pred, doc_tgt):
      single_predicted += qry_pred
      single_tgt += qry_tgt
  best_f1 = f1_score(single_predicted, single_tgt)
  best_acc = accuracy_score(single_predicted, single_tgt)
  print(f'Total Best f1_score:{round(best_f1, 3)}, accuracy:{round(best_acc, 3)}')

In [ ]:
num_sent_range = range(1, 1001, 1)
all_lead_list = lead_list_creation_valid(num_sent_range, tgt_list_valid)
show_statistics_valid(all_lead_list, tgt_list_valid, [i for i in num_sent_range])

Goal1, threshold: 258, f1_score: 0.036, accuracy: 0.63
Goal2, threshold: 258, f1_score: 0.039, accuracy: 0.633
Goal3, threshold: 270, f1_score: 0.148, accuracy: 0.617
Goal4, threshold: 278, f1_score: 0.071, accuracy: 0.606
Goal5, threshold: 660, f1_score: 0.084, accuracy: 0.222
Goal6, threshold: 260, f1_score: 0.032, accuracy: 0.627
Goal7, threshold: 493, f1_score: 0.115, accuracy: 0.366
Goal8, threshold: 461, f1_score: 0.151, accuracy: 0.403
Goal9, threshold: 278, f1_score: 0.137, accuracy: 0.604
Goal10, threshold: 459, f1_score: 0.06, accuracy: 0.385
Goal11, threshold: 270, f1_score: 0.119, accuracy: 0.625
Goal12, threshold: 260, f1_score: 0.141, accuracy: 0.626
Goal13, threshold: 493, f1_score: 0.12, accuracy: 0.367
Goal14, threshold: 496, f1_score: 0.064, accuracy: 0.35
Goal15, threshold: 258, f1_score: 0.091, accuracy: 0.633
Goal16, threshold: 479, f1_score: 0.067, accuracy: 0.367
Goal17, threshold: 471, f1_score: 0.117, accuracy: 0.382


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.096, accuracy:0.497


In [ ]:
# Test

In [ ]:
lead_length_list = [258, 258, 270, 278, 660, 260, 493, 461, 278, 459, 270, 260, 493, 496, 258, 479, 471]
def lead_list_creation_test(lead_length_list, tgt_list_test):
  lead_list_test = []
  for tgt_list in tgt_list_test:
    lead_list_by_qry = []
    for i in range(17):
      if lead_length_list[i] <= len(tgt_list[i]):
        one_list = [1] * lead_length_list[i]
        zero_list = [0] * (len(tgt_list[i]) - lead_length_list[i])
      else:
        one_list = [1] * len(tgt_list[i])
        zero_list = []
      lead_list_by_qry.append(one_list + zero_list)
    lead_list_test.append(lead_list_by_qry)
  return lead_list_test

In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_test(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')

In [ ]:
lead_length_list = [258, 258, 270, 278, 660, 260, 493, 461, 278, 459, 270, 260, 493, 496, 258, 479, 471]
all_lead_list = lead_list_creation_test(lead_length_list, tgt_list_test)
show_statistics_test(all_lead_list, tgt_list_test)

Goal1, f1_score: 0.015, accuracy: 0.618
Goal2, f1_score: 0.008, accuracy: 0.615
Goal3, f1_score: 0.058, accuracy: 0.588
Goal4, f1_score: 0.036, accuracy: 0.589
Goal5, f1_score: 0.065, accuracy: 0.209
Goal6, f1_score: 0.012, accuracy: 0.605
Goal7, f1_score: 0.094, accuracy: 0.356
Goal8, f1_score: 0.12, accuracy: 0.389
Goal9, f1_score: 0.079, accuracy: 0.577
Goal10, f1_score: 0.05, accuracy: 0.376
Goal11, f1_score: 0.044, accuracy: 0.591
Goal12, f1_score: 0.087, accuracy: 0.603
Goal13, f1_score: 0.148, accuracy: 0.375
Goal14, f1_score: 0.053, accuracy: 0.341
Goal15, f1_score: 0.045, accuracy: 0.616
Goal16, f1_score: 0.037, accuracy: 0.347
Goal17, f1_score: 0.082, accuracy: 0.366
Total, f1_score: 0.067, accuracy: 0.48
